# Recreating cassava disease detection using an SVM

In [1]:
# --- Cell 1: Imports and File Paths ---
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set file paths for the E8 Trial (Trial 3)
UNINFECTED_PATH = 'data/TME204-Patch_E4_E6_E8_28DPI_Dataset/E8-TME204/E8_TME204_28dpi_Dec_16_2020/Uninfected/E8_TME204_28dpi_Dec_16_2020_Uninfected_11_06_23_33.mat'
UCBSV_PATH = 'data/TME204-Patch_E4_E6_E8_28DPI_Dataset/E8-TME204/E8_TME204_28dpi_Dec_16_2020/UCBSV/E8_TME204_28dpi_Dec_16_2020_UCBSV_11_06_23_36.mat'

print("Libraries imported and paths defined.")

Libraries imported and paths defined.


In [2]:
# --- A Simple, Direct Test ---
import scipy.io
import numpy as np

# We will use the UCBSV file for this test as we have seen its structure in MATLAB
UCBSV_PATH = 'data/TME204-Patch_E4_E6_E8_28DPI_Dataset/E8-TME204/E8_TME204_28dpi_Dec_16_2020/UCBSV/E8_TME204_28dpi_Dec_16_2020_UCBSV_11_06_23_36.mat'

print(f"Loading file: {UCBSV_PATH}")

# Load the data. squeeze_me=True simplifies the nested structure.
mat_data = scipy.io.loadmat(UCBSV_PATH, squeeze_me=True)

# The 'Patch' key contains an array of struct-like objects.
# In this file, it's an array of 18 structs.
all_scans = mat_data['Patch']

# Access the first leaf scan's struct from the array
first_scan_struct = all_scans[0]

# From that single struct, access the 'mean_values' field.
# This is the pre-calculated 9x14 matrix of features we saw in MATLAB.
mean_values_for_first_leaf = first_scan_struct['mean_values']

print("\n--- Verification Successful ---")
print(f"Type of extracted data: {type(mean_values_for_first_leaf)}")
print(f"Shape of extracted data: {mean_values_for_first_leaf.shape}")
print("\nMean values for the 9 patches of the first leaf:")
print(mean_values_for_first_leaf)

Loading file: data/TME204-Patch_E4_E6_E8_28DPI_Dataset/E8-TME204/E8_TME204_28dpi_Dec_16_2020/UCBSV/E8_TME204_28dpi_Dec_16_2020_UCBSV_11_06_23_36.mat

--- Verification Successful ---
Type of extracted data: <class 'numpy.ndarray'>
Shape of extracted data: ()

Mean values for the 9 patches of the first leaf:
[[ 18.26475694  17.18098958  17.80555556  25.25694444  24.59548611
   24.61458333  25.88975694  23.25651042  23.6171875   21.74262153
   19.94270833  24.29123264  24.03776042 126.12022569]
 [ 19.30425347  18.21831597  18.87239583  25.87934028  25.06987847
   25.06336806  26.22135417  23.58810764  24.19704861  22.32855903
   20.63671875  24.69661458  24.30555556 123.49262153]
 [ 18.93142361  17.78862847  18.63888889  26.63758681  26.12847222
   25.92491319  27.57161458  24.67751736  25.06857639  23.02039931
   21.27647569  25.41840278  25.09635417 128.41710069]
 [ 16.72395833  16.09722222  15.64453125  20.52300347  19.93663194
   20.49869792  21.06814236  18.90625     19.37717014  17.

In [3]:
# --- Cell 2: Process the INFECTED (UCBSV) File ---
print(f"Processing file: {UCBSV_PATH}")

# Load the .mat file for the infected class. 
# 'squeeze_me=True' helps simplify the nested data structure loaded by scipy.
mat_data_ucbsv = scipy.io.loadmat(UCBSV_PATH, squeeze_me=True)

# Access the 'Patch' field, which contains an array of structs, one for each leaf scan.
all_scans_ucbsv = mat_data_ucbsv['Patch']

# Initialize a list to store the feature vectors for this class.
ucbsv_features = []

# Iterate through each leaf scan's struct.
for scan_struct in all_scans_ucbsv:
    # Extract the pre-calculated 'mean_values' from the struct.
    # .item() is used to retrieve the numpy array from its 0-d container.
    mean_vals_matrix = scan_struct['mean_values'].item()
    
    # The matrix is 9x14, where each row is a feature vector for one patch.
    # Add all 9 feature vectors to the list for this class.
    ucbsv_features.extend(mean_vals_matrix)

# Print a confirmation with the total number of patches processed.
print(f"Successfully processed {len(ucbsv_features)} patches from the UCBSV file.")

Processing file: data/TME204-Patch_E4_E6_E8_28DPI_Dataset/E8-TME204/E8_TME204_28dpi_Dec_16_2020/UCBSV/E8_TME204_28dpi_Dec_16_2020_UCBSV_11_06_23_36.mat
Successfully processed 162 patches from the UCBSV file.


In [4]:
# --- Cell 3: Process the UNINFECTED File ---
print(f"Processing file: {UNINFECTED_PATH}")

# Load the .mat file for the uninfected class.
mat_data_uninfected = scipy.io.loadmat(UNINFECTED_PATH, squeeze_me=True)

# Access the 'Patch' field, containing an array of structs for each leaf scan.
all_scans_uninfected = mat_data_uninfected['Patch']

# Initialize a list to store the feature vectors for this class.
uninfected_features = []

# Iterate through each leaf scan's struct.
for scan_struct in all_scans_uninfected:
    # Extract the pre-calculated 'mean_values' from the struct.
    # .item() is used to retrieve the numpy array from its 0-d container.
    mean_vals_matrix = scan_struct['mean_values'].item()
    
    # The matrix is 9x14, where each row is a feature vector for one patch.
    # Add all 9 feature vectors to the list for this class.
    uninfected_features.extend(mean_vals_matrix)

# Print a confirmation with the total number of patches processed.
print(f"Successfully processed {len(uninfected_features)} patches from the Uninfected file.")

Processing file: data/TME204-Patch_E4_E6_E8_28DPI_Dataset/E8-TME204/E8_TME204_28dpi_Dec_16_2020/Uninfected/E8_TME204_28dpi_Dec_16_2020_Uninfected_11_06_23_33.mat
Successfully processed 162 patches from the Uninfected file.


In [5]:
# --- Cell 4: Combine Data and Finalize ---

# Create numerical labels for each class (0 for uninfected, 1 for infected).
uninfected_labels = [0] * len(uninfected_features)
ucbsv_labels = [1] * len(ucbsv_features)

# Combine the features and labels from both files into final numpy arrays.
X = np.array(uninfected_features + ucbsv_features)
y = np.array(uninfected_labels + ucbsv_labels)

# Print the final shapes to confirm the dataset is correctly assembled.
print(f"\nProcessing complete.")
print(f"Final feature matrix X shape: {X.shape}")
print(f"Final label vector y shape: {y.shape}")

# For easier inspection, create a pandas DataFrame.
# Define column names based on the wavelengths from the research paper.
wavelengths = [395, 415, 470, 528, 532, 550, 570, 585, 590, 610, 625, 640, 660, 880]
columns = [f'wl_{wl}' for wl in wavelengths]
df = pd.DataFrame(X, columns=columns)
df['label'] = y

# Display the top and bottom of the final DataFrame to verify its contents.
print("\n--- Final Dataset Head ---")
display(df.head())
print("\n--- Final Dataset Tail ---")
display(df.tail())


Processing complete.
Final feature matrix X shape: (324, 14)
Final label vector y shape: (324,)

--- Final Dataset Head ---


,wl_395,wl_415,wl_470,wl_528,wl_532,wl_550,wl_570,wl_585,wl_590,wl_610,wl_625,wl_640,wl_660,wl_880,label
0,19.703559,18.059462,16.894531,27.229601,26.131076,24.549045,28.139323,24.205295,25.448785,22.825955,20.521267,22.292969,21.752170,107.310764,0
1,17.602865,16.409722,15.253038,25.079427,24.518229,22.747396,26.451389,23.542969,24.570312,22.351997,19.956597,20.965278,20.433160,92.058160,0
2,18.036892,16.857639,15.898438,25.563368,24.887587,23.215712,26.603299,23.303819,24.248264,21.845920,19.694010,21.176649,20.752170,96.516059,0
3,19.746962,17.976128,17.099826,27.790365,26.640191,24.930122,28.705295,24.440104,25.649306,22.872396,20.459201,22.441406,22.033420,110.956163,0
4,18.998264,17.991319,16.598958,26.419271,25.569010,24.216146,27.644097,25.355903,25.124566,22.956597,20.721354,22.669705,22.957899,110.958333,0



--- Final Dataset Tail ---


,wl_395,wl_415,wl_470,wl_528,wl_532,wl_550,wl_570,wl_585,wl_590,wl_610,wl_625,wl_640,wl_660,wl_880,label
319,18.399414,17.708984,17.836914,25.652344,25.061523,24.613281,26.552734,24.635742,24.850586,23.331055,21.224609,23.934570,23.666016,110.960938,1
320,16.695312,16.180664,16.449219,23.918945,23.214844,22.682617,24.471680,21.979492,22.477539,20.779297,19.007812,21.821289,21.415039,101.840820,1
321,17.804688,16.208008,17.116211,23.941406,23.110352,23.979492,24.686523,24.301758,22.676758,21.613281,19.517578,24.284180,25.083984,136.611328,1
322,18.318359,16.760742,17.037109,23.227539,22.474609,23.113281,23.661133,22.624023,21.751953,20.505859,18.880859,22.994141,23.229492,118.534180,1
323,18.838867,16.842773,16.708008,23.207031,22.743164,22.996094,23.920898,21.993164,22.583984,20.851562,19.057617,22.569336,22.363281,110.278320,1
